In [11]:
import rouskinhf
import pandas as pd
import numpy as np


In [47]:

training_dataset_names = ['rnacentral_synthetic', 'ribo500-blast', 'bpRNA-1m', 'pri_miRNA', 'human_mRNA']
testing_dataset_names = [ 'PDB', 'viral_fragments', 'lncRNA', 'archiveII_filtered']

def dataset_to_df(dataset_name):
    return pd.DataFrame.from_dict(rouskinhf.get_dataset(dataset_name), orient='index').reset_index()

train_set = pd.concat({name: dataset_to_df(name) for name in training_dataset_names})
train_set = train_set.drop_duplicates('sequence')
test_set = pd.concat({name: dataset_to_df(name) for name in testing_dataset_names})

In [48]:
train_set

index  \
rnacentral_synthetic 0      RF02271.fa.csv_1   
                     1      RF02271.fa.csv_2   
                     2      RF02271.fa.csv_3   
                     3      RF02271.fa.csv_4   
                     4      RF02271.fa.csv_5   
...                                      ...   
human_mRNA           1451  ENSG00000284791.2   
                     1452  ENSG00000286522.2   
                     1453  ENSG00000287363.4   
                     1454  ENSG00000288920.1   
                     1455  ENSG00000291309.1   

                                                                    sequence  \
rnacentral_synthetic 0     UCUUAUGGAUGAGCAAAGAAAGUGGUUUCUUGAGAUGGAAUUUACU...   
                     1     UCUUAUGGGUGAGCAAGAAAAGUGGUUUCUUGAGAUGGAAGCUACU...   
                     2     UCUUACGGAUGAGCAAAGAAAGUGGUUUCUUGAACAGAAACUACUC...   
                     3     GUCUACAGCAAUGUUGCCCUGAACAAGCCCGAUCCCAUCUGAUUGU...   
                     4     UGAGACUCCCUCACCGCCCUGGGUGUGCCUGAUCCCAUAUGAUCUC...   
...                                                                      ...   
human_mRNA           1451  GGGGGUGCUCGGCGUGCUGUCCCUGCUGGUGCUUUGCGGGGUCCUG...   
                     1452  GGGUGCAGGCAAGCUUUUCUGUGGUUUUGCCAUGGCUCGUACUAAA...   
                     1453  GGGAGACACCGUCAUGCAAGGGGGUGAAGGCUCCACACUCGUCCCG...   
                     1454  GGGUUUUCCGCGCUACCUGCAGAGGGGUCCAUACGGCGUUGUUCUG...   
                     1455  GGGGGUCAGUCUUCCUCUGGGCCCUGCUGUGAGGGCCUUUCCCGCC...   

                                                                   structure  \
rnacentral_synthetic 0     [[26, 39], [18, 47], [7, 59], [6, 60], [17, 48...   
                     1     [[23, 42], [29, 36], [26, 39], [22, 43], [20, ...   
                     2     [[4, 61], [3, 62], [29, 35], [13, 51], [20, 44...   
                     3     [[93, 108], [94, 107], [39, 76], [83, 116], [4...   
                     4     [[5, 111], [30, 43], [71, 102], [21, 52], [20,...   
...                                                                      ...   
human_mRNA           1451  [[29, 139], [366, 474], [176, 277], [10, 162],...   
                     1452  [[240, 304], [223, 233], [280, 297], [249, 254...   
                     1453  [[197, 220], [25, 154], [91, 120], [198, 219],...   
                     1454  [[614, 631], [480, 484], [260, 279], [517, 542...   
                     1455  [[408, 416], [257, 320], [409, 415], [258, 319...   

                                                                         dms  \
rnacentral_synthetic 0                                                   NaN   
                     1                                                   NaN   
                     2                                                   NaN   
                     3                                                   NaN   
                     4                                                   NaN   
...                                                                      ...   
human_mRNA           1451  [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...   
                     1452  [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...   
                     1453  [-1000.0, -1000.0, -1000.0, 0.7213, -1000.0, 0...   
                     1454  [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...   
                     1455  [-1000.0, -1000.0, -1000.0, -1000.0, -1000.0, ...   

                          shape  
rnacentral_synthetic 0      NaN  
                     1      NaN  
                     2      NaN  
                     3      NaN  
                     4      NaN  
...                         ...  
human_mRNA           1451   NaN  
                     1452   NaN  
                     1453   NaN  
                     1454   NaN  
                     1455   NaN  

[359472 rows x 5 columns]

In [49]:
intersection

,level_0_x,level_1_x,sequence,structure_x,dms_x,shape_x,level_0_y,level_1_y,structure_y,dms_y,shape_y,source
0,rnacentral_synthetic,RF02271.fa.csv_107,GUCUACGACCAUACCACGUUGAAAGCACGACAUCUCGUCCGAUCUG...,"[[3, 114], [4, 113], [96, 104], [24, 51], [67,...",NaN,NaN,archiveII_filtered,5s_Dirofilaria-immitis-1,"[[0, 117], [1, 116], [2, 115], [3, 114], [4, 1...",NaN,NaN,rnacentral_synthetic contains archiveII_filtered
1,rnacentral_synthetic,RF02271.fa.csv_6961,ACAUACGACCAUAGGGUGUGGAAAACAGGGCUUCCCGUCCGCUCAG...,"[[79, 97], [8, 110], [0, 118], [19, 58], [84, ...",NaN,NaN,archiveII_filtered,5s_Emericella-nidulans-6,"[[0, 118], [1, 117], [2, 116], [3, 115], [4, 1...",NaN,NaN,rnacentral_synthetic contains archiveII_filtered
2,rnacentral_synthetic,RF02271.fa.csv_11174,CAUAACGGUCAUAGCGGAGGAGUUACAUCCGAUCCCAUCCCGAUCU...,"[[30, 46], [19, 58], [72, 100], [29, 47], [18,...",NaN,NaN,archiveII_filtered,5s_Methanococcus-maripaludis-1,"[[0, 112], [2, 111], [3, 110], [4, 109], [5, 1...",NaN,NaN,rnacentral_synthetic contains archiveII_filtered
3,bpRNA-1m,bpRNA_PDB_307,GGGAUCACAAGUAGGACUUCGGUCCGAAUACAGAUCUC,"[[0, 37], [1, 36], [2, 35], [3, 34], [4, 33], ...",NaN,NaN,PDB,2KHY-2D,"[[0, 37], [2, 35], [3, 34], [4, 33], [5, 32], ...",NaN,NaN,bpRNA-1m contains PDB
4,bpRNA-1m,bpRNA_PDB_342,GGGACCUUCCAAGUCUC,"[[0, 16], [1, 15], [2, 14], [3, 13], [4, 12]]",NaN,NaN,PDB,2LBL-2D,"[[0, 16], [1, 15], [2, 14], [3, 13], [4, 12]]",NaN,NaN,bpRNA-1m contains PDB
...,...,...,...,...,...,...,...,...,...,...,...,...
283,bpRNA-1m,bpRNA_PDB_312,GGACGGUGGCAGCACUGUCUAGCUGCGGGCAUUAGACUGGAAAACU...,"[[1, 18], [2, 17], [3, 16], [4, 15], [5, 14], ...",NaN,NaN,PDB,2KRL-2D,"[[1, 18], [2, 17], [3, 16], [4, 15], [5, 14], ...",NaN,NaN,bpRNA-1m contains PDB
284,bpRNA-1m,bpRNA_PDB_19,GGCGACGGUGUAAAAAUCUCGCC,"[[0, 22], [1, 21], [2, 20], [3, 19], [4, 18], ...",NaN,NaN,PDB,1BVJ-2D,"[[0, 22], [1, 21], [2, 20], [3, 19], [4, 18], ...",NaN,NaN,bpRNA-1m contains PDB
285,bpRNA-1m,bpRNA_PDB_78,GGGCGUGCCC,"[[0, 9], [1, 8], [2, 6]]",NaN,NaN,PDB,1IDV-2D,"[[0, 9], [1, 8], [2, 7]]",NaN,NaN,bpRNA-1m contains PDB
286,bpRNA-1m,bpRNA_PDB_373,GGUUUCUUUUUAGUGAUUUUUCCAAACCCCUUUGUGCAAAAAUCAUUA,"[[0, 27], [1, 26], [2, 25], [3, 24], [4, 23], ...",NaN,NaN,PDB,2M8K-2D,"[[0, 27], [1, 26], [2, 25], [3, 24], [4, 23], ...",NaN,NaN,bpRNA-1m contains PDB


In [23]:
print("{}/{} sequences of the testing set are in the training set".format(len(set(test_set['sequence']).intersection(set(train_set['sequence']))), len(test_set['sequence'])))

288/761 sequences of the testing set are in the training set


In [31]:
intersection = pd.merge(train_set.reset_index(), test_set.reset_index(), on='sequence', how='inner')

import plotly.express as px

intersection['source'] = intersection['level_0_x'] + ' contains ' + intersection['level_0_y']

fig = px.pie(intersection, names='source', title='Sources of the sequences in the intersection of the training and testing sets')
fig.show()

In [46]:
train_set.index

MultiIndex([('rnacentral_synthetic',  'RF02271.fa.csv_1'),
            ('rnacentral_synthetic',  'RF02271.fa.csv_2'),
            ('rnacentral_synthetic',  'RF02271.fa.csv_3'),
            ('rnacentral_synthetic',  'RF02271.fa.csv_4'),
            ('rnacentral_synthetic',  'RF02271.fa.csv_5'),
            ('rnacentral_synthetic',  'RF02271.fa.csv_6'),
            ('rnacentral_synthetic',  'RF02271.fa.csv_7'),
            ('rnacentral_synthetic',  'RF02271.fa.csv_8'),
            ('rnacentral_synthetic',  'RF02271.fa.csv_9'),
            ('rnacentral_synthetic', 'RF02271.fa.csv_10'),
            ...
            (          'human_mRNA', 'ENSG00000283329.2'),
            (          'human_mRNA', 'ENSG00000283706.2'),
            (          'human_mRNA', 'ENSG00000283755.2'),
            (          'human_mRNA', 'ENSG00000283980.1'),
            (          'human_mRNA', 'ENSG00000284638.2'),
            (          'human_mRNA', 'ENSG00000284791.2'),
            (          'human_mRNA', 'EN

In [41]:
# tun blast

!mkdir -p blast

def write_fasta(filename, df):
    with open(filename, 'w') as f:
        for i, seq in enumerate(sequences):
            f.write(">{}\n{}\n".format(i, seq))
            
    
# make a blast database
!makeblastdb -in blast/train.fasta -dbtype nucl -out blast/train




Building a new DB, current time: 05/08/2024 14:20:57
New DB name:   /Users/yvesmartin/src/supermodels-data/blast/train
New DB title:  blast/train.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
BLAST options error: File blast/train.fasta does not exist
